<a href="https://colab.research.google.com/github/informatics-isi-edu/facebase-ml-exec/blob/main/notebooks/VGG19_Diagnosis_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 

This notebook applied a pre-trained model to a dataset specified in the configuration file and uploads the labels to the catalog.  The ROC curve is also calculated and uploaded.

In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "facebase-ml"))

In [2]:
import json
import os
from facebase_ml.facebase_ml import FaceBaseML
import pandas as pd
from pathlib import Path, PurePath
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2024-04-30 19:14:27.123100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 19:14:27.878093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "fb-ml" #@param
host = 'ml.facebase.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-04-30 19:14:34,112 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 19:14:34,113 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.
# Change to your username on the VM

configuration_rid="58-TW70"      # Configuration file for this run.  Needs to be changed for each execution.

In [5]:
FB = FaceBaseML(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)
print(f" Initializing model version: {FB.version}")

2024-04-30 19:14:37,429 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 19:14:37,430 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


 Initializing model version: 1.1.0.post1+git.d2faa8b8


In [ ]:
# @title Initiate an Execution
configuration_records = FB.execution_init(configuration_rid=configuration_rid)
input_dataset = configuration_records.bag_paths[0] # Assumes that the configuration file only specifies one dataset.
configuration_records.model_dump()

2024-04-30 20:40:54,916 - INFO - File [/data/lizhiwei/FaceBaseML_working/Execution_Metadata/Execution_Config-MusMorph_train_10pct.json] transfer successful. 0.74 KB transferred. Elapsed time: 0:00:00.000075.
2024-04-30 20:40:54,916 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/FaceBaseML_working/Execution_Metadata/Execution_Config-MusMorph_train_10pct.json]
2024-04-30 20:40:54,935 - INFO - Configuration validation successful!
2024-04-30 20:40:56,375 - INFO - Checking payload consistency. This can take some time for large bags with many payload files...
2024-04-30 20:40:56,899 - INFO - Attempting to resolve remote file references from /data/58-TC6A_dbe4619ed2511dd8fae878901edefa38419604b264c4c5d3ad32e9d3c5f5944c/Dataset_58-TC6A/fetch.txt.
2024-04-30 20:41:00,339 - INFO - Attempting GET from URL: https://ml.facebase.org/hatrac/facebase/data/fb3/3-KB0J/3-THKG/img/B6_11.mnc:32AZYH5OOUQW3TJRPROGHPETJQ
2024-04-30 20:41:04,640 - INFO - File [/data/58-TC6A_dbe4619ed2511dd

In [ ]:
train_base_dir = configuration_records.bag_paths[0]
valid_base_dir = configuration_records.bag_paths[1]
test_base_dir = configuration_records.bag_paths[2]

biosample_filename = 'data/biosample.csv'
genotype_filename = 'data/genotype.csv'
# output_filename = 'data/mapped_file.csv'

final_df, mapped_file = FB.join_and_save_csv(train_base_dir, biosample_filename, genotype_filename, output_filename) #  does it output files?

Algorithm was trained on cropped images, so take the raw images and bounding boxes and apply, storing the results in the working directory.

Import the actual model code and then run against the input dataset specified in the configuration file.  

In [ ]:
# @title Execute Proecss algorithm (Test model)
from facebase_ml_tools.models.some_file import #some model

with FB.execution(execution_rid=configuration_records.execution_rid) as exec:
  output_path = FB.execution_assets_path/Path("Model_Prediction")


In [ ]:
# @title Plot ROC.


Add the new lables to the catalog using the provided diagnosis tage for this execution.  Also upload any additional assets that were produced by this execution..

In [ ]:
# @title Save Diagnosis


In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = FB.execution_upload(configuration_records.execution_rid, False)
